# Visualize 3D Time-Series of Meshes

## Set-up and Imports

In [1]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)

Working directory:  /home/nmiolane/code/my28brains


In [2]:
import glob

import numpy as np
import trimesh

import my28brains.viz as viz

REGISTERED_MESHES_DIR = os.path.join(os.getcwd(), "data", "registered_meshes")
print("REGISTERED_MESHES_DIR: ", REGISTERED_MESHES_DIR)

REGISTERED_MESHES_DIR:  /home/nmiolane/code/my28brains/data/registered_meshes


In [3]:
hemisphere = "left"
ply_suffix = f"ashs/{hemisphere}_lfseg_corr_usegray_CT_LQ.ply"
string_base = os.path.join(REGISTERED_MESHES_DIR, f"sub-01/ses-**/{ply_suffix}")
paths = sorted(glob.glob(string_base))

print(f"Found {len(paths)} ply files.")
print(paths)

Found 27 ply files.
['/home/nmiolane/code/my28brains/data/registered_meshes/sub-01/ses-02/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/sub-01/ses-03/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/sub-01/ses-04/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/sub-01/ses-05/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/sub-01/ses-06/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/sub-01/ses-07/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/sub-01/ses-08/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/sub-01/ses-09/ashs/left_lfseg_corr_usegray_CT_LQ.ply', '/home/nmiolane/code/my28brains/data/registered_meshes/sub-01/ses-10/ashs/left_lfseg_corr_usegray_C

In [4]:
geodesic = []
base_mesh = trimesh.load(paths[0])
faces = np.array(base_mesh.faces)

# source = (base_mesh.vertices, base_mesh.faces)
# target_mesh = trimesh.load(paths[4])
# target = (target_mesh.vertices, target_mesh.faces)

for path in paths[:5]:  ## TODO REPLACE
    mesh = trimesh.load(path)
    vertices = np.array(mesh.vertices)
    geodesic.append(vertices)

geodesic = tuple(geodesic)

In [5]:
viz.plotGeodesic(geodesic, faces)

[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [6]:
def plotGeodesic(
    geod,
    F,
    source=None,
    target=None,
    file_name=None,
    offsetstep=2.5,
    stepsize=2,
    axis=[0, 0, 1],
    angle=-1 * np.pi / 2,
):
    """Plot geodesic evolution after symmetric or asymmetric matching with the H2 metric and varifold relaxation.

    Input:
        - geod: geodesic path [tuple with tuple[k]=vertices of k^th surface in the geodesic stored as an nVx3 ndarray]
        - F: faces for the mesh structure of the surfaces on the geodesic path [nFx3 ndarray]
        - source [tuple with source[0]=vertices, source[1]=faces, default=None]
        - target [tuple with target[0]=vertices, target[1]=faces, default=None]
        - file_name: specified path for saving geodesic mesh [string, default=None]
        - offsetstep: spacing between different geodesics on the plot [default=2.5]
        - stepsize: spacing within a geodesic on the plot [default=2]
        - axis: axis of rotation for each individual surface in the geodesic [default=[0,0,1]]
        - angle: angle of rotation [default=-pi/2]
    Output:
        - Plot of geodesic with source (left), geodesic path (middle) and target (right)
        - file_name.ply file containing geodesic mesh information (optional)
    """